### RNN
1. Sequence Data : 순서가 존재하는 데이터
    - 주로 시계열(Time Series) 데이터 
2. 작동원리 
    - 첫 번째 은닉층의 값이 다음번 다시 해당 은닉층의 입력으로 들어가는 것 
    - 정리 
        - 은닉층 노드들은 어떤 초기값을 가지고 계산 
        - 첫 번째 입력값이 들어온 t = 0 시점에서 입력값과 초기값의 조합으로 은닉층의 값들이 계산 
        - t = 1 시점에서 새로 들어온 입력값과 t = 0 시점에서 계산된 은닉층의 값과 조합으로 결과값이 다시 계산 
        - 이러한 과정을 반복 
        - 기존의 역전파와는 다르게 순환 신경망은 계산에 사용된 시점의 수에 영향을 받음 ex) t = 0에서 t = 2까지 계산에 사용되면 그 시간 전체를 역전파 
        - t = 2 시점에서 발생한 손실은 t = 2,1,0 시점에 전부 영향을 주고 계속 내려가는 방식 
        - 실제 업데이트할 때 가중치에 대해 시점별 기울기를 다 더해서 한번에 업데이트 

### Create Model

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim 
import numpy as np

In [61]:
n_hidden = 35 
lr = 0.001
epochs = 1000

string = 'hello pytorch. how long can a rnn cell remember?'
chars = 'abcdefghijklmnopqrstuvwxyz ?!.,:;01'
char_list = [i for i in chars]
n_letters = len(char_list)

In [62]:
def string_to_onehot(string):
    start = np.zeros(shape = len(char_list), dtype = int)
    end = np.zeros(shape = len(char_list), dtype = int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape = len(char_list), dtype = int)
        zero[idx] = 1
        start = np.vstack([start,zero])
    output = np.vstack([start, end])
    return output

# one_hot 인코딩을 해주는 함수 
# np.vstack : 배열을 세로로 결합할 때 사용 
# start와 end는 왜 사용할까 사용하면 길이가 2개가 더 길어지는데 

In [63]:
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

# one_hot 인코딩이 된 것을 다시 돌려주는 함수 

In [64]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # layer를 정의해줌 
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
        
    def forward(self, input, hidden):
        hidden = self.act_fn(self.i2h(input) + self.h2h(hidden))
        output = self.i2o(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
        # hidden layer의 초기값을 1로 지정 
    
rnn = RNN(n_letters, n_hidden, n_letters)

In [65]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = lr)

### Train

1. 학습하고자 하는 문장을 one-hot벡터로 변환한 넘파이 array를 다시 torch.Tensor로 변경 
2. 자료형은 torch.FloatTensor로 지정 
3. 이렇게 하면 start_token + 문장 + end_token으로 구성된 매트릭스가 생성 (아 ~~~ 그래서 start랑 end랑 나눴구나)


In [66]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    # 매 epoch마다 기울기 초기화 
    total_loss = 0
    hidden = rnn.init_hidden()
    
    for j in range(one_hot.size()[0]-1):
        # 실제 데이터는 48개이므로 48번 돌기위해서 
        input_ = one_hot[j:j+1,:]
        target = one_hot[j+1]
        
        
        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        total_loss += loss
        input_ = output
        
    total_loss.backward()
    optimizer.step()
    
    if i % 10 ==0:
        print(total_loss)


tensor(2.5141, grad_fn=<AddBackward0>)
tensor(1.6394, grad_fn=<AddBackward0>)
tensor(1.3345, grad_fn=<AddBackward0>)
tensor(1.2275, grad_fn=<AddBackward0>)
tensor(1.1483, grad_fn=<AddBackward0>)
tensor(1.0734, grad_fn=<AddBackward0>)
tensor(1.0046, grad_fn=<AddBackward0>)
tensor(0.9369, grad_fn=<AddBackward0>)
tensor(0.8702, grad_fn=<AddBackward0>)
tensor(0.8063, grad_fn=<AddBackward0>)
tensor(0.7463, grad_fn=<AddBackward0>)
tensor(0.6914, grad_fn=<AddBackward0>)
tensor(0.6419, grad_fn=<AddBackward0>)
tensor(0.5976, grad_fn=<AddBackward0>)
tensor(0.5572, grad_fn=<AddBackward0>)
tensor(0.5195, grad_fn=<AddBackward0>)
tensor(0.4838, grad_fn=<AddBackward0>)
tensor(0.4499, grad_fn=<AddBackward0>)
tensor(0.4177, grad_fn=<AddBackward0>)
tensor(0.3875, grad_fn=<AddBackward0>)
tensor(0.3597, grad_fn=<AddBackward0>)
tensor(0.3345, grad_fn=<AddBackward0>)
tensor(0.3117, grad_fn=<AddBackward0>)
tensor(0.2911, grad_fn=<AddBackward0>)
tensor(0.2725, grad_fn=<AddBackward0>)
tensor(0.2556, grad_fn=<A

In [67]:
start = torch.zeros(1, len(char_list))
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ''
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data)
        input_ = output
print(output_string)
#결과는 별로 좋지 못하다.

hello pytollrrow ol  ow ro  ol  ol rol rl  ol eo


### 결과 
    - 문장이 길어지게 되면 학습이 안되는 문제 즉, rnn은 한계가 있음 